In [7]:
import notebookjs
import glob
from notebookjs import execute_js
import random
from sklearn.calibration import calibration_curve
import sklearn
import shap
import pandas as pd
import numpy as np
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix


sys.path.insert(0, '../core/')
from calibrate import Calibrate

## User Simulation

In [8]:
X,y = shap.datasets.adult(display=True)

le = sklearn.preprocessing.LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])
X = pd.get_dummies(X, columns=["Workclass", "Marital Status", "Occupation", "Relationship", "Race", "Country"])

scaler = sklearn.preprocessing.StandardScaler()
train_col_sacle = X[['Age','Education-Num','Hours per week']]
train_scaler_col = scaler.fit_transform(train_col_sacle)
train_scaler_col = pd.DataFrame(train_scaler_col,columns=train_col_sacle.columns)
X['Age'] = train_scaler_col['Age']
X['Education-Num'] = train_scaler_col['Education-Num']
X['Hours per week'] = train_scaler_col['Hours per week']

X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

clf = MLPClassifier(random_state=42, max_iter=100).fit(X_train, y_train)
y_preds = clf.predict_proba(X_valid)

# OHE labels
ohe = sklearn.preprocessing.OneHotEncoder()

y_valid_labels = ohe.fit_transform(y_valid.reshape(-1,1)).toarray()

In [10]:
dataset = X_valid[['Age', 'Education-Num', 'Sex', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Workclass_ ?']]
c = Calibrate(data=dataset)

## adding models
c.add_model(y_preds, y_valid_labels, 'MLPClassifier')

## plotting
c.calibrate()